In [1]:
%load_ext autotime

time: 265 µs (started: 2022-11-18 18:38:42 +00:00)


# Cargar modelos

In [2]:
# imporar librerias
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch.utils.data as data
from torchvision import transforms 
from torch import nn, optim
import torch.optim as optim
from torchvision.models import resnet101
import urllib.request
import cv2

from ignite.metrics import ClassificationReport
from ignite.engine import *
from ignite.handlers import *
from ignite.metrics import *
from ignite.utils import *
from ignite.contrib.metrics.regression import *
from ignite.contrib.metrics import *

from genericpath import exists

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


time: 1.22 s (started: 2022-11-18 18:38:42 +00:00)


In [3]:
# basic python and ML Libraries
import os
import random
import numpy as np
import pandas as pd

# for ignoring warnings
import warnings
warnings.filterwarnings('ignore')

# We will be reading images using OpenCV
import cv2

# matplotlib for visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# torchvision libraries
import torch
import torchvision
from torchvision import transforms as torchtrans  
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# helper libraries
from engine import train_one_epoch, evaluate
import utils
import transforms as T

# for image augmentations
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

time: 91.3 ms (started: 2022-11-18 18:38:43 +00:00)


In [4]:
from torchvision.models import resnext101_32x8d
resnext1 = resnext101_32x8d(pretrained=True)
resnext2 = resnext101_32x8d(pretrained=True)

time: 2.84 s (started: 2022-11-18 18:38:44 +00:00)


In [5]:
# Cargar backbone resnet y resnext
class MultilabelClassifier1(nn.Module):
    def __init__(self, n_classes, pretrain_model):
        super().__init__()
        self.pretrain_model = pretrain_model
        self.model_wo_fc = nn.Sequential(*(list(self.pretrain_model.children())[:-1]))

        self.classes = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=2048, out_features=n_classes)
        )

    def forward(self, x):
        x = self.model_wo_fc(x)
        x = torch.flatten(x, 1)

        return {
            'label': self.classes(x)
        }

time: 1.03 ms (started: 2022-11-18 18:38:46 +00:00)


In [6]:
# cargar modelo para cara y n° domicilio
device = 'cuda'
#device = 'cpu'
PATH_1 =  '/home/jupyter/Score/Modelos/mlc_model_4.pth'
ml_model_1 = MultilabelClassifier1(3, resnext1).to(device)
ml_model_dict_1 = torch.load(PATH_1, map_location=torch.device(device))
ml_model_1.load_state_dict(ml_model_dict_1['model_state_dict'])
#torch.save(ml_model_1.load_state_dict(ml_model_dict_1['model_state_dict']), '/home/jupyter/Score/Modelos/mlc_state_dict_4.pt')
#ml_model_1.eval()

<All keys matched successfully>

time: 4.23 s (started: 2022-11-18 18:38:46 +00:00)


In [7]:
ml_model_dict_1['validation loss']

2.648120448645789

time: 3.12 ms (started: 2022-11-18 18:38:51 +00:00)


In [8]:
# obtener epoca óptima
optimal_epoch = ml_model_dict_1['epoch']
print(optimal_epoch)

6
time: 1.28 ms (started: 2022-11-18 18:38:51 +00:00)


In [9]:
# Cargar backbone -> para modelo 'mlc_model_baseline_data_revisada_1_2.pth'
class MultilabelClassifier2(nn.Module):
    def __init__(self, n_classes, pretrain_model):
        super().__init__()
        self.pretrain_model = pretrain_model
        self.model_wo_fc = nn.Sequential(*(list(self.pretrain_model.children())[:-1]))

        self.classes = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=2048, out_features=3)
        )

    def forward(self, x):
        x = self.model_wo_fc(x)
        x = torch.flatten(x, 1)

        return {
            'label': self.classes(x)
        }

time: 1.44 ms (started: 2022-11-18 18:38:51 +00:00)


In [10]:
# cargar modelo para etiqueta del producto
device = 'cuda'
#device = 'cpu'
PATH_2 = '/home/jupyter/Score/Modelos/mlc_model_baseline_data_revisada_1_2.pth'
ml_model_2 = MultilabelClassifier2(3, resnext2).to(device)
ml_model_dict_2 = torch.load(PATH_2, map_location=torch.device(device))
ml_model_2.load_state_dict(ml_model_dict_2['model_state_dict'])
#torch.save(ml_model_2.load_state_dict(ml_model_dict_2['model_state_dict']), '/home/jupyter/Score/Modelos/mlc_state_dict_baseline_data_revisada_1_2.pt')
#ml_model_2.eval()

<All keys matched successfully>

time: 789 ms (started: 2022-11-18 18:38:51 +00:00)


In [11]:
ml_model_dict_2['validation loss']

2.0754197467943256

time: 2.75 ms (started: 2022-11-18 18:38:52 +00:00)


In [12]:
# obtener epoca óptima
optimal_epoch = ml_model_dict_2['epoch']
print(optimal_epoch)

10
time: 1.2 ms (started: 2022-11-18 18:38:52 +00:00)


In [13]:
# REFERENCIAR!
# función de ayuda para cargar modelo
def get_object_detection_model(num_classes):
  # load a model pre-trained on COCO
  model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
  # get number of input features for the classifier
  in_features = model.roi_heads.box_predictor.cls_score.in_features
  # replace the pre-trained head with a new one
  model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes) 
  return model

time: 927 µs (started: 2022-11-18 18:38:52 +00:00)


In [14]:
# cargar modelo detector de objetos
device = 'cuda'
#device = 'cpu'
model_path = '/home/jupyter/Score/Modelos/model_cf_12y3_1.pth'
num_classes = 2
od_model = get_object_detection_model(num_classes)
od_model_dict = torch.load(model_path, map_location=torch.device(device))
od_model.load_state_dict(od_model_dict['model_state_dict'])
#torch.save(od_model.load_state_dict(od_model_dict['model_state_dict']), '/home/jupyter/Score/Modelos/state_dict_cf_12y3_1.pt')

<All keys matched successfully>

time: 842 ms (started: 2022-11-18 18:38:52 +00:00)


# Obtener data

In [15]:
from google.cloud import bigquery

client = bigquery.Client()

time: 10.5 ms (started: 2022-11-18 18:38:52 +00:00)


In [16]:
# Query para obtener la data de 3 días antes, considerando que el enlace de la foto, 
# el nombre del transportista y su rut, no sean nulos


sql = """
SELECT DISTINCT transport_ord_id as SOC, i.url as url, shipment.plate_num as plate_num, 
provider.doc_id as provider_id, 
provider.doc_verify_digit as provider_verify_digit,
provider.name as provider_name, driver.doc_id as driver_id, 
driver.doc_verify_digit as driver_verify_digit,
driver.name as driver_name, driver.last_name as driver_last_name,
DATETIME(event_crte_tmst, 'America/Santiago') as event_crte_tmst, dfl_crte_tmst
FROM 
`tc-sc-bi-bigdata-corp-tsod-dev.image_recognition.btd_scha_fal_trmg_api_transport_order_temp`,
unnest(image) as i
 
WHERE
  i.url is not null
  and provider.name is not null
  and provider.doc_id is not null
  and DATE(event_crte_tmst, 'America/Santiago') = current_date() - 3
"""

df_images = client.query(sql).to_dataframe()

print(len(df_images))

53168
time: 3.11 s (started: 2022-11-18 18:38:52 +00:00)


In [18]:
# eliminar filas con urls duplicados
df_images = df_images.drop_duplicates(['url'])
# subir imagenes a bucket
import datetime
import numpy as np
import pandas as pd
import cv2
import urllib
import urllib.request
from multiprocessing import cpu_count
from multiprocessing.pool import ThreadPool
from google.cloud import bigquery
from google.cloud import storage
date = datetime.datetime.now()    
def get_data(url):
    try:
        url_str = url.split('/')[-1]
        url_open = urllib.request.urlopen(url)
        
        image_cv = np.asarray(bytearray(url_open.read()), dtype="uint8")
        image = cv2.imdecode(image_cv, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        img_str = cv2.imencode('.png', image)[1].tostring()
        
        storage_client = storage.Client()
        bucket = storage_client.bucket('pod_images')
        # el cero es para distinguir 
        #blob = bucket.blob(f'{date.year}{date.month}{date.day}/{url_str}')
        blob = bucket.blob(f'{2022}{11}0{17}/{url_str}')
        blob.upload_from_string(img_str)
    except:
        # ver el link que tira error
        print(url)
#urls = df_images.loc[:,'url'].iloc[:10000]
urls = df_images.loc[:,'url'].iloc[:1000]
cpus = cpu_count()
results = ThreadPool(cpus-1).imap_unordered(get_data, urls)

time: 48 ms (started: 2022-11-18 18:39:16 +00:00)


In [ ]:
# resetear indices
df_images = df_images.reset_index()
# eliminar columna index
df_images = df_images.drop(['index'], axis = 1)
print(len(df_images))

In [27]:
import requests
import time
from multiprocessing import cpu_count
from multiprocessing.pool import ThreadPool

time: 556 µs (started: 2022-11-18 17:56:20 +00:00)


Pruebas para leer enlaces mas rapido

In [28]:
#inputs_df = df_images.iloc[0:100].url
#inputs_list = df_images.iloc[0:10].url.to_list()
#inputs_array = df_images.iloc[0:10].url.to_numpy()

time: 318 µs (started: 2022-11-18 17:56:21 +00:00)


In [47]:
from google.cloud import storage
import cv2
from tempfile import TemporaryFile
import numpy as np
import datetime

date=datetime.datetime.now()

#nombre carpeta
#folder = str(date.year) + str(date.month) + str(date.day)
folder = '20221118'

# inicialización
client = storage.Client()
bucket = client.get_bucket('pod_images')
#blobs = client.list_blobs(bucket)
blobs = bucket.list_blobs(prefix=folder)
# generar lista de fotos
blob_list = list(blobs)
print(len(blob_list))

47198
time: 4.2 s (started: 2022-11-18 18:33:19 +00:00)


In [30]:
from google.cloud import storage
import cv2
from tempfile import TemporaryFile
import numpy as np


# función para leer imágenes desde el bucket
def read_image_bucket(blob_object):
    filename = blob_object.name
    imagename = filename.split('/')[1]

    client = storage.Client()
    
    #bucket = client.get_bucket('image_recognition_images')
    bucket = client.get_bucket('pod_images')
    blob = bucket.get_blob(filename)
    
    try:
        b = blob.download_as_bytes()
        image_cv = np.asarray(bytearray(b), dtype="uint8")
        image = cv2.imdecode(image_cv, cv2.IMREAD_COLOR)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        area = image.shape[0]*image.shape[1]
        url = 'https://prdadessacorptrl.blob.core.windows.net/cl-images/' + imagename
        #print(image.shape)
        #print(area)
    except:
        image = np.zeros((480,480,3), np.uint8)
        area = image.shape[0]*image.shape[1]
        url = 'https://prdadessacorptrl.blob.core.windows.net/cl-images/' + imagename
        
    return(image, area, url)

time: 1.14 ms (started: 2022-11-18 17:37:31 +00:00)


In [31]:
# funcion para trabajar en paralelo
#200.000 imagenes aprox 3 horas
# se podria agregar un log cada vez que lee 1000 images y
# además ir guardando la lista, de manera de que si se corta
# el proceso, se puede retomar desde donde que quedo
def open_parallel_bucket(args):
    cpus = cpu_count()
    #results = ThreadPool(cpus - 1).imap_unordered(read_image_bucket, args)
    results = ThreadPool(cpus-1).imap_unordered(read_image_bucket, args)
    import numpy as np
    lista_base = []
    for result in results:
        lista_base.append(result)
    return(lista_base)

time: 733 µs (started: 2022-11-18 17:37:32 +00:00)


In [32]:
# leer fotos
t_ini = time.time()
l_bucket = open_parallel_bucket(blob_list)

time: 1min 30s (started: 2022-11-18 17:37:32 +00:00)


# Generar dataloaders

In [24]:
# Dataset class para generar los batches

class PredictionDataset(Dataset):
    """Photos prediction dataset."""

    def __init__(self, lista_fotos, normalization = 'mlc', device = 'cuda'):
        """
        Args:
            lista_fotos (list): Lista con varias fotos concatenadas y sus areas
            normalization (string): Indica si se debe implementar la transformación
            de normalización para el clasificador multi-etiqueta ('mlc') o para el 
            detector de objetos ('ob')
            device (string): Indica qué dispositivo se quiere usar: 'cuda' o 'cpu'
        
        Returns:
            pro_img (tensor): Retorna la imagen procesada como tensor lista para la predicción.
        """
        self.lista_fotos = lista_fotos
        self.normalization = normalization
        self.device = device
        # transformación para clasificador
        self.mlc_normalization = transforms.Compose([transforms.ToPILImage(),
                                                 transforms.Resize((224, 224)),
                                                 transforms.ToTensor(),
                                                 transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                                                      std=[0.229, 0.224, 0.225]),
                                                 ])
    
    def __len__(self):
        return len(self.lista_fotos)

    def __getitem__(self, idx):
        foto = self.lista_fotos[idx][0]
        area = self.lista_fotos[idx][1]
        url = self.lista_fotos[idx][2]
        # aplicar normalizacion
        if self.normalization == 'mlc':
            pro_img  = self.mlc_normalization(foto)
            #print(t_img.shape)
            #pro_img = t_img.unsqueeze(dim = 0)
            #print(pro_img.shape)
        
        elif self.normalization == 'od':
            t_img_0 = transform.resize(foto, (480, 480))
            pro_img = torch.Tensor(t_img_0).permute(2,0,1)

        # retornar imagen procesada
        return pro_img, area, url

time: 2.39 ms (started: 2022-11-17 18:40:20 +00:00)


In [25]:
# se generan batch para clasificador multi-etiqueta y detector de objetos
prediction_set_mlc = PredictionDataset(lista_fotos = l_bucket, normalization = 'mlc', device = 'cuda')
prediction_set_od = PredictionDataset(lista_fotos = l_bucket, normalization = 'od', device = 'cuda')

# para aumentar más el batch_size hay que agregar más memoria
predictionLoader_mlc = DataLoader(prediction_set_mlc, batch_size=32, num_workers=2,
                         shuffle = False)
predictionLoader_od = DataLoader(prediction_set_od, batch_size=32, num_workers=2,
                         shuffle = False)

time: 1.63 ms (started: 2022-11-17 18:40:20 +00:00)


# Función de score

In [26]:
# Librerias
import pandas as pd
import numpy as np
import os
import io
import cv2
import urllib
import math
import urllib.request

time: 898 µs (started: 2022-11-17 18:40:20 +00:00)


In [27]:
def bbox_function(b, area_img):
    l =list(range(b.shape[0]))
    # recorrer cajas y calcular inter-area para cada combinación
    indice = 0
    interArea = 0
    for i in itertools.combinations(l, r=2):
        # obtener coordenadas de la inter-area
        x0 = max(b[i[0]][0], b[i[1]][0])
        y0 = max(b[i[0]][1], b[i[1]][1])
        x1 = min(b[i[0]][2], b[i[1]][2])
        y1 = min(b[i[0]][3], b[i[1]][3])

        # calcular inter-area
        dif_x = x0-x1
        dif_y = y0-y1
        # se verifica que las esquinas de la interArea esten bien ubicadas 
        if dif_x < 0 and dif_y < 0:
            interArea += dif_x*dif_y
            ##interArea += abs(x0-x1)*abs(y0-y1)
    
    # sumar areas de cada bbox
    area_total_bbox = 0
    for box in b:
        # calcular area de cada bbox
        area_bbox = abs(box[0]-box[2])*abs(box[1]-box[3])
        # calcular area total de bbox
        area_total_bbox += area_bbox

    # calcular la union de las areas de cada bbox
    union = area_total_bbox - interArea

    # calcular contexto
    if torch.is_tensor(union):
        contexto = union.item()/area_img
    
    else:
        contexto = union/area_img
    
    return contexto

time: 2.26 ms (started: 2022-11-17 18:40:20 +00:00)


In [28]:
import itertools
#from google.colab.patches import cv2_imshow
import cv2
# supuesto: tabla con enlaces a cada fotografía

def score(predictionLoader_mlc, predictionLoader_od, classificator_1, classificator_2, 
          detector = od_model,pesos = {'w_prod': 0.4, 'w_notface': 0.1,'w_label': 0.3, 'w_num': 0.1, 
                                       'w_contx': 0.2},
          thresholds = {'t_prod': 0.5, 't_face': 0.5, 't_label': 0.5, 't_num': 0.5, 't_ctx_down': 0.2, 
                        't_ctx_up': 0.65}, device = 'cuda'):
    """
    Args:
        classificator_1 (modelo): clasificador multi-etiqueta para detectar la etiqueta del paquete
        classificator_2 (modelo): clasificador multi-etiqueta para detectar la cara y el domicilio
        detector (modelo): detector de objetos para identificar el paquete y su bbox respectivo
        pesos (dict): diccionario con valores para cada peso, es decir,
        w_prod (product), w_notface (without face), w_label (product label), 
        w_num (address number) y w_contx (context)
        thresholds = diccionario con valores de umbral para cada criterio, es decir,
        t_prod (product), t_face (face detector), t_label (product label) y 
        t_num (address number)
        
    Obs: los pesos deben sumar 1 para todos los criterios menos el del numero de domicilio. Este
        ultimo corresponde a un beneficio de +0.1 si es que aparece en la fotografía.

    Returns:
        result_data (dataFrame): cada una de las columnas del dataFrame corresponde a la predicción
        de cada criterio sobre cierto umbral, los scores (confianza del modelo) de cada criterio y 
        la nota de la foto (score).
    """

    #pesos = {'w_prod': 0.4, 'w_notface': 0.1,'w_label': 0.3, 'w_num': 0.1, 'w_contx': 0.2}
    
    #thresholds = {'t_prod': 0.5, 't_face': 0.5, 't_label': 0.5, 't_num': 0.5, 't_ctx_down': 0.2, 
    #              't_ctx_up': 0.65}
    
    #device = 'cuda'
    # evaluar con gpu o cpu 
    device = torch.device(device)
    #classificator_1 = ml_model_1 
    #classificator_2 = ml_model_2
    #detector = od_model
    
    classificator_1.to(device)
    classificator_1.eval()  
    classificator_2.to(device)
    classificator_2.eval()           
    detector.to(device)
    detector.eval()
    df_base = pd.DataFrame(columns = ['url','paquete', 's_paquete', 'etiqueta_producto',
                                          's_etiqueta_producto', 'sin_rostro', 's_sin_rostro',
                                          'numero_domicilio', 's_numero_domicilio', 'contexto',
                                          'ctx_value', 'score'])
    with torch.no_grad():
        for i, image in enumerate(zip(predictionLoader_mlc, predictionLoader_od)):
            # se obtiene la imagen de cada dataLoader
            image_mlc = image[0][0]
            image_od = image[1][0]
            
            # obtener areas y urls de las imagenes
            area_img = image[0][1]
            #img_url = np.asarray(image[0][2]).reshape(-1,1)
            img_url = list(image[0][2])
    
            # CLASIFICACION MULTI-ETIQUETA
            #---------------------------clasificador 1--------------------------------
            label_base = classificator_1(image_mlc.to(device))
            #---------------------------clasificador 2--------------------------------
            label_etiqueta = classificator_2(image_mlc.to(device))
            # obtener score
            score_etiqueta = torch.sigmoid(label_etiqueta['label']).squeeze()
            score = torch.sigmoid(label_base['label']).squeeze()
            ##print('score:',score.shape)
            # obtener predicción etiqueta
            s_etiqueta = score_etiqueta[:,0]
            etiqueta = torch.where(s_etiqueta >= thresholds['t_label'], 1, 0)
            # obtener predicción domicilio
            s_domicilio = score[:,1]
            domicilio = torch.where(s_domicilio >= thresholds['t_num'], 1, 0)
            # obtener predicción cara
            s_cara = score[:,2]
            s_no_cara = 1-s_cara
            no_cara = torch.where(s_no_cara >= thresholds['t_face'], 1, 0)
            
            etiqueta = etiqueta.reshape(-1,1).to('cpu').numpy()
            s_etiqueta = s_etiqueta.reshape(-1,1).to('cpu').numpy()
            domicilio = domicilio.reshape(-1,1).to('cpu').numpy()
            s_domicilio = s_domicilio.reshape(-1,1).to('cpu').numpy()
            no_cara = no_cara.reshape(-1,1).to('cpu').numpy()
            s_no_cara = s_no_cara.reshape(-1,1).to('cpu').numpy()
            
            #-----------------------------------------------------------------------------
            # DETECTOR DE OBJETOS
            od_prediction = od_model(image_od.to(device))
            ##print(od_prediction)
            dataFrame = pd.DataFrame(od_prediction).to_dict(orient="list")
            #print(dataFrame['boxes'])
            #print(torch.Tensor(dataFrame['boxes']))
            list_of_lists = list(map(lambda x: x.tolist(), dataFrame['boxes']))
            array_of_arrays1= np.array(list(map(lambda x: x.to('cpu').numpy(), dataFrame['boxes'])))
            array_of_arrays2= np.array(list(map(lambda x: x.to('cpu').numpy(), dataFrame['scores'])))
            #bla = list(map(lambda x: torch.where(x >= 0.5, 1, 0), dataFrame['scores']))
            bla2 = np.array(list(map(lambda x: np.where(x.to('cpu').numpy()>=thresholds['t_prod'],1,
                                                        0).reshape(-1,1),
                                     dataFrame['scores'])))
            #print(array_of_arrays1)
            #print(bla2)
            producto = array_of_arrays1*bla2
            #print(producto)
            producto_wz = np.array(list(map(lambda x: x[~np.all(x == 0, axis=1)], producto)))
            contexto = np.array(list(map(lambda x,y: bbox_function(x,y), producto_wz, area_img)))
            contexto = contexto.reshape(-1,1)
            ctx_value = np.where(((contexto >= thresholds['t_ctx_down']) & (contexto <= thresholds['t_ctx_up'])),
                                 1, 0) 
            #print(ctx_value)
            #print(array_of_arrays*array_of_arrays)
            #print(array_of_arrays2)
            #print(np.where(array_of_arrays2.all() < 0.5, 0, array_of_arrays2))
            #print(list(map(lambda x: max(x), dataFrame['boxes'])))
            s_paquete = torch.tensor(list(map(lambda x: max(x, default=0), dataFrame['scores'])))
            s_paquete = s_paquete.reshape(-1,1)
            #printtorch.tensor(dataFrame['scores'])
            #print(scores.reshape(-1,1).to('cpu').numpy())
            paquete = torch.where(s_paquete >= thresholds['t_prod'], 1, 0)
            #############listos con score!!#################3
            # que pasa si no se puede leer el url?
            # que pasa si la imagen no tiene bbox? -> funciona todo bien
            # falta enlace: corecto o incorrecto
            
            result = np.concatenate([paquete, s_paquete, etiqueta, s_etiqueta, no_cara, s_no_cara,
                                    domicilio, s_domicilio, contexto, ctx_value], axis = 1)
            
            ##print(result)
            df = pd.DataFrame(result, columns=['paquete','s_paquete','etiqueta_producto', \
                                                           's_etiqueta_producto', 'sin_rostro', 's_sin_rostro',\
                                                           'numero_domicilio', 's_numero_domicilio', 'contexto',\
                                                           'ctx_value'])
            
            pesos = {'w_prod': 0.4, 'w_notface': 0.1,'w_label': 0.3, 'w_num': 0.1, 'w_contx': 0.2}
            df['score'] = df.paquete*pesos['w_prod'] + df.etiqueta_producto*pesos['w_label'] + \
            df.sin_rostro*pesos['w_notface'] + df.numero_domicilio*pesos['w_num'] + df.ctx_value*pesos['w_contx']
            #FALTA GUARDAR EL ENLACE Y NOTIFICAR CUANDO UN ENLACE ESTA ERRONEO
            #df['url'] = img_url
            df.insert(loc=0, column='url', value=img_url)
            # concatenar a df anterior
            df_base = pd.concat((df_base,df), ignore_index= True)
    return df_base

time: 5.71 ms (started: 2022-11-17 18:40:20 +00:00)


In [29]:
df_base = score(predictionLoader_mlc, predictionLoader_od, ml_model_1 , ml_model_2, 
          detector = od_model,pesos = {'w_prod': 0.4, 'w_notface': 0.1,'w_label': 0.3, 'w_num': 0.1, 
                                       'w_contx': 0.2},
          thresholds = {'t_prod': 0.5, 't_face': 0.6491, 't_label': 0.3, 't_num': 0.6, 't_ctx_down': 0.2, 
                            't_ctx_up': 0.65}, device = 'cuda')
t_fin = time.time()
delta_time = t_fin-t_ini

time: 4.39 s (started: 2022-11-17 18:40:20 +00:00)


# Generación nueva tabla BigQuery

In [34]:
def table_generation(df_images, df_base, delta_time):
    inner_merged = pd.merge(df_images, df_base, on=["url"])
    df_base_copy1 = inner_merged.copy()
    # normalización
    #agregar indice
    largo_dataset = len(df_base_copy1)
    df_base_copy1['index'] = df_base_copy1.index
    # Cambiar tipo de dato de "score" a float
    df_base_copy1["score"] = pd.to_numeric(df_base_copy1["score"])
    # Seleccionar SOC que tiene mayor "score"
    idx_max_score = df_base_copy1.groupby(['SOC'])['score'].transform(max) == df_base_copy1['score']
    ##df_base_copy2 = df_base_copy1[idx_max_score].drop_duplicates(['SOC'])
    # Agrupar por SOC y dejar indice
    ##df_base_copy2 = df_base_copy1[['SOC', 'index', 'score']].groupby(['SOC'], sort = False).score.max()
    # Agrupar por RUT, normalizar los nombres y dejar indice
    #.agg(lambda x:x.value_counts().index[0])
    df_base_copy3 = df_base_copy1[['provider_name', 'provider_id']].groupby(['provider_id'], sort = False)['provider_name'].transform('first').to_frame()
    df_base_copy3['index'] = df_base_copy1['index']
    # "merge" del dataset normalizado y el dataset original, con respecto al indice
    df_base_copy4 = pd.merge(df_base_copy3, df_base_copy1, how='left', on='index')
    # "merge" del dataset agrupado por SOC y el dataset anterior, con respecto al indice
    ##df_base_final  = pd.concat([df_base_copy2, df_base_copy4], axis=1, join="inner")
    df_base_final = df_base_copy1[idx_max_score].drop_duplicates(['SOC'])
    ##df_base_final = pd.merge(df_base_copy2, df_base_copy4, how='left', on='index')
    # arreglar fechas
    print(df_base_final['dfl_crte_tmst'])
    df_base_final['dfl_crte_tmst'] = df_base_final['dfl_crte_tmst'].dt.tz_localize(None)
    print(df_base_final['dfl_crte_tmst'])
    df_base_final['event_crte_tmst'] = df_base_final['event_crte_tmst'].dt.tz_localize(None)
    # guardar largo del dataset
    new_df_base_final = df_base_final.assign(len_data = largo_dataset)
    # guardar tiempo de ejecución
    new_df_base_final = new_df_base_final.assign(execution_time_model = delta_time)
    
    return new_df_base_final

time: 1.85 ms (started: 2022-11-17 18:42:15 +00:00)


In [35]:
new_df_base_final = table_generation(df_images, df_base, delta_time)

0   2022-11-14 03:00:11.278000+00:00
4   2022-11-14 03:00:20.229000+00:00
9   2022-11-14 03:06:58.763000+00:00
Name: dfl_crte_tmst, dtype: datetime64[ns, UTC]
0   2022-11-14 03:00:11.278
4   2022-11-14 03:00:20.229
9   2022-11-14 03:06:58.763
Name: dfl_crte_tmst, dtype: datetime64[ns]
time: 27.8 ms (started: 2022-11-17 18:42:15 +00:00)


# Guardar tablas

In [32]:
def tables_save(new_df_base_final, n_r, project_id = 'tc-sc-bi-bigdata-corp-tsod-dev', dataset = 'image_recognition', 
                full_table = 'prediction_table', random_table = 'random_table', 
                random_table_backup  = 'random_table_backup', prediction_if_exists = 'append',
           random_if_exists = 'replace', random_backup_if_exists = 'append'):
    # generar tabla aleatoria
    df_copy = new_df_base_final.copy()
    #df_filtrado = df_copy[(df_copy.enlace != 'incorrecto')]
    df_copy=df_copy.assign(paquete_em="", etiqueta_em="", domicilio_em="", rostro_em="")
    import pandas_gbq
    # ubicación de destino para tabla procesada
    destination_full_table = dataset + '.' + full_table
    # anexar tabla de predicciones
    pandas_gbq.to_gbq(new_df_base_final, destination_full_table, project_id=project_id, if_exists = prediction_if_exists)
    print("Se almacenó exitosamente tabla de predicciones")
    # generar tabla aleatoria
    try:
        # seleccionar pequeño conjunto de datos aleatorios de cada sample
        # intentar generar tabla aleatoria
        random_sample = df_copy.sample(n_r, frac=None, replace=False, weights=None, random_state=None)
        # destino de tabla aleatoria
        destination_random_table = dataset + '.' + random_table
        # anexar tabla aleatoria a tabla generada durante el dia
        pandas_gbq.to_gbq(random_sample, destination_random_table, project_id=project_id, 
                          if_exists = random_if_exists)
        print("Se almacenó exitosamente sub-tabla aleatoria")
        # anexar tabla aleatoria, con todas las tablas aleatorias generadas en dias anteriores
        # para tenerlas de respaldo
        destination_random_table_backup = dataset + '.' + random_table_backup
        pandas_gbq.to_gbq(random_sample, destination_random_table_backup, project_id=project_id, 
                          if_exists = random_backup_if_exists)  
        print("Se almacenó exitosamente sub-tabla aleatoria al backup")
    except:
        print("No se pudo generar tabla aleatoria (muy pocas filas)")

time: 1.69 ms (started: 2022-11-16 20:30:05 +00:00)


In [33]:
tables_save(new_df_base_final, n_r = 200, project_id = 'tc-sc-bi-bigdata-corp-tsod-dev', dataset = 'image_recognition', 
                full_table = 'prediction_table_optimizado_final', random_table = 'random_table_optimizado_final', 
                random_table_backup  = 'random_table_backup_optimizado_final',  prediction_if_exists = 'append',
           random_if_exists = 'replace', random_backup_if_exists = 'append')

100%|██████████| 1/1 [00:00<00:00, 12122.27it/s]


Se almacenó exitosamente tabla de predicciones


100%|██████████| 1/1 [00:00<00:00, 16008.79it/s]


Se almacenó exitosamente sub-tabla aleatoria


100%|██████████| 1/1 [00:00<00:00, 15252.01it/s]

Se almacenó exitosamente sub-tabla aleatoria al backup
time: 17.3 s (started: 2022-11-16 20:30:05 +00:00)
